# Criando Modelos Para Definir o Plano de Telefone

## Importando Dados

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
users_behavior = pd.read_csv('/datasets/users_behavior_upd.csv')

In [3]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   calls        3214 non-null   float64
 1   minutes      3214 non-null   float64
 2   messages     3214 non-null   float64
 3   mb_used      3214 non-null   float64
 4   is_ultimate  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
users_behavior.head()

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Separando os conjuntos de treino, validação e teste

In [5]:
# Dividindo as características e o alvo
features = users_behavior.drop(['is_ultimate'], axis=1)
target = users_behavior['is_ultimate']

In [6]:
# Separando o grupo de teste
x, features_test, y, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

In [7]:
# Separando o grupo de treinamento e o de validação
features_train, features_valid, target_train, target_valid = train_test_split(x, y, test_size=0.25, random_state=12345)

## Criando Modelos

### Árvore de Decisão

In [8]:
decision_tree_best_model = None
decision_tree_best_score = 0
decision_tree_best_depth = 0

for depth in range(1, 21):
    decision_tree_model = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    decision_tree_model.fit(features_train, target_train)
    decision_tree_score = decision_tree_model.score(features_valid, target_valid)
    if decision_tree_score > decision_tree_best_score:
        decision_tree_best_model = decision_tree_model
        decision_tree_best_score = decision_tree_score
        decision_tree_best_depth = depth

print('Melhor Acurácia:', decision_tree_best_score)
print('Melhor profundidade:', decision_tree_best_depth)

Melhor Acurácia: 0.776049766718507
Melhor profundidade: 7


In [30]:
# Acurácia no Conjunto de teste
print("Acurácia do modelo de árvore de decisão no conjunto de validação:", decision_tree_best_model.score(features_test, target_test))

Acurácia do modelo de árvore de decisão no conjunto de validação: 0.7900466562986003


### Floresta Aleatória

In [10]:
random_forest_best_score = 0
random_forest_best_est = 0
random_forest_best_model = None
for est in range(1, 21): # escolha o intervalo para o hiperparâmetro
    random_forest_model = RandomForestClassifier(random_state=54321, n_estimators=est)
    random_forest_model.fit(features_train, target_train)
    random_forest_score = random_forest_model.score(features_valid, target_valid) # cálculo da acurácia do conjunto de validação
    if random_forest_score > random_forest_best_score:
        random_forest_best_score = random_forest_score
        random_forest_best_est = est
        random_forest_best_model = random_forest_model

print(f"A acurácia do melhor modelo no conjunto de validação (n_estimators = {random_forest_best_est}): {random_forest_best_score}")

A acurácia do melhor modelo no conjunto de validação (n_estimators = 20): 0.7931570762052877


In [29]:
# Acurácia no Conjunto de teste
print("Acurácia do modelo de floresta aleatória no conjunto de validação:", random_forest_best_model.score(features_test, target_test))

Acurácia do modelo de floresta aleatória no conjunto de validação: 0.7900466562986003


### Regressão Logística

In [17]:
def logistic_regression_fitting(solver):    
    logistic_regression_model = LogisticRegression(random_state=54321, solver=solver)
    logistic_regression_model.fit(features_train, target_train)
    logistic_regression_score_train = logistic_regression_model.score(features_train, target_train) # calculo da acurácia no conjunto de treinamento
    logistic_regression_score_valid = logistic_regression_model.score(features_valid, target_valid) # calculo da acurácia no conjunto de validação

    return logistic_regression_model, logistic_regression_score_train, logistic_regression_score_valid

In [24]:
best_logistic_regression_model = None

# Usando o solver 'liblinear'
best_logistic_regression_model, lr_score_train_liblinear, lr_score_valid_liblinear = logistic_regression_fitting('liblinear')
print("Acurácia do modelo de regressão logística no conjunto de treinamento:", lr_score_train_liblinear)
print("Acurácia do modelo de regressão logística no conjunto de validação:", lr_score_valid_liblinear)

Acurácia do modelo de regressão logística no conjunto de treinamento: 0.703838174273859
Acurácia do modelo de regressão logística no conjunto de validação: 0.6967340590979783


In [25]:
# Usando o solver 'lbfgs'
lr_model, lr_score_train_lbfgs, lr_score_valid_lbfgs = logistic_regression_fitting('lbfgs')
print("Acurácia do modelo de regressão logística no conjunto de treinamento:", lr_score_train_lbfgs)
print("Acurácia do modelo de regressão logística no conjunto de validação:", lr_score_valid_lbfgs)

Acurácia do modelo de regressão logística no conjunto de treinamento: 0.7510373443983402
Acurácia do modelo de regressão logística no conjunto de validação: 0.7262830482115086


In [27]:
# Escolha do melhor modelo
if lr_score_valid_lbfgs > lr_score_valid_liblinear:
    best_logistic_regression_model = lr_model


# Acurácia no Conjunto de teste
print("Acurácia do modelo de regressão logística no conjunto de teste:", best_logistic_regression_model.score(features_test, target_test))

Acurácia do modelo de regressão logística no conjunto de teste: 0.7589424572317263


## Conclusão

Para o conjunto de dados apresentados, onde o modelo deve sugerir o melhor plano (Smart ou Ultra) conforme as características apresentadas (chamadas, minutos usados, quantidade de mensagens e consumo de internet), dividimos os dados na proporção 3:1:1, sendo 60% para o conjunto de treinamento, 20% para o conjunto de validação e 20% para o conjunto de teste, pois não possuimos um conjunto de teste separado.<br>
O problema é um problema de classificação, ou plano Smart ou plano Ultra, sendo assim utilizamos 3 algoritmos de aprendizado para definir qual seria o acurado, lembrando que o limite para a acurácia é de 0.75. <br>
O modelo da Árvore de Decisão atingiu uma acurácia de 0.776 no conjunto de validação e 0.79 no conjunto de Teste.<br>
O modelo de Floresta Aleatória atingiu uma acurácia de 0.793 no conjunto de validação e 0.79 no conjunto de Teste.<br>
O modelo de Regressão Logística atingiu uma acurácia de 0.726 no conjunto de validação e 0.759 no conjunto de Teste.<br>
Portanto, podemos descartar o modelo de Regressão Logística para o nosso caso devido a baixa acurácia no conjunto de validação. Entre o modelo de Árvore de Decisão e Floresta Aleatória há pouca diferença entre as acurácias com vantagem para o metodo de Floresta Aleatória, mas o modelo de Árvore de Decisão é mais rápido.